In [1]:
import pandas as pd
import numpy as np
import time
import os 
from pandas import Series, DataFrame
from tqdm.notebook import tqdm

dir_ = '../data/'
file_name = 'filter_track_5_user_50.pkl'

In [2]:
lh = pd.read_pickle(os.path.join(dir_, file_name))
unique_users = lh['uid'].unique()
unique_user = len(unique_users)
unique_tracks = lh['tid'].unique()
unique_track = len(unique_tracks)

print(unique_user)
print(unique_track)

218
39137


In [3]:
lh[:5]

,uid,tid,count,tags
0,0,0,2,[1]
2,0,2,2,"[2, 1]"
7,0,7,4,"[0, 8]"
8,0,8,1,[0]
10,0,10,5,"[1, 12]"


In [4]:
track_keys = {}

for track in unique_tracks:
    track_keys[track] = len(track_keys)
    
user_keys = {}

for user in unique_users:
    user_keys[user] = len(user_keys)

lh['tid']= lh['tid'].map(track_keys)
lh['uid']= lh['uid'].map(user_keys)
lh[:5]

,uid,tid,count,tags
0,0,0,2,[1]
2,0,1,2,"[2, 1]"
7,0,2,4,"[0, 8]"
8,0,3,1,[0]
10,0,4,5,"[1, 12]"


In [5]:
threshold = []
for i in tqdm(lh['uid'].unique()):
    user = lh[lh['uid'] == i]
    user = user.sort_values(by=['count'])
    t = []
    for j in range(19):
        v = user.iloc[len(user)//20*(j+1)]['count']
        t.append(v)
    threshold.append(t)
threshold[0]

[1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 4, 4, 5, 6, 6, 8, 11, 15]

In [6]:
start = time.time()
def to_rating(df):  
    rating = 0
    for i in range(18,-1,-1):
        if df['count'] <= threshold[df['uid']][i]:
            continue
        else:
            rating = i+1
            break
    return 0.25*(rating+1)
        
lh['rating'] = lh.apply(to_rating , axis = 1)

end = time.time()
print(end - start)

36.40635561943054


In [7]:
lh[:5]

,uid,tid,count,tags,rating
0,0,0,2,[1],1.50
2,0,1,2,"[2, 1]",1.50
7,0,2,4,"[0, 8]",3.00
8,0,3,1,[0],0.25
10,0,4,5,"[1, 12]",3.50


In [8]:
lh.to_pickle(os.path.join(dir_, 'normalized_to_rating_' + file_name))